In [173]:
# 0. Load libraries #

import numpy as np
import pandas as pd
import os, time, warnings, random, gc, pickle
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.svm import SVC
from sklearn.preprocessing import LabelBinarizer, LabelEncoder, OrdinalEncoder, OneHotEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, cross_val_predict, GridSearchCV, train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier, VotingClassifier
from sklearn.metrics import accuracy_score, f1_score
from sklearn.inspection import permutation_importance
from xgboost import XGBClassifier

pd.set_option('display.max_columns', 300)
pd.set_option('display.max_rows', 300)
pd.set_option('mode.chained_assignment', None)
pd.set_option('display.expand_frame_repr', False)
warnings.filterwarnings('ignore')

# Load custom pre-processing functions:

def draw_histograms(df, variables, n_rows, n_cols):
    # stolen from https://stackoverflow.com/questions/29530355/plotting-multiple-histograms-in-grid
    fig=plt.figure()
    for i, var_name in enumerate(variables):
        ax=fig.add_subplot(n_rows,n_cols,i+1)
        df[var_name].hist(bins=10,ax=ax)
        ax.set_title(var_name+" Distribution")
    fig.tight_layout()  
    plt.show()


def fillna_mp_i1(df_train, df_test, df_pred, num_features, cat_features, num_fill='median', cat_fill='mode'):
    """This function speeds up filling missing values for 3 main datasets using different imputation methods.
    Later may replace it with some subclass.
    Example: fillna_mp_i1(X_train, X_test, X_pred, num_cols, cat_cols)"""
    # set df_pred to None if it does not exist
    if not ((cat_fill=='mode') and (num_fill=='median')):
        print ('Imputation method not Implemented yet!')
        return None
    
    df_train[num_features] = df_train[num_features].fillna(value=df_train[num_features].median())
    df_test[num_features] = df_test[num_features].fillna(value=df_train[num_features].median())
    df_train[cat_features] = df_train[cat_features].fillna(value=df_train[cat_features].mode().iloc[0])
    df_test[cat_features] = df_test[cat_features].fillna(value=df_train[cat_features].mode().iloc[0])
    if (df_pred is not None):
        df_pred[num_features] = df_pred[num_features].fillna(value=df_train[num_features].median())
        df_pred[cat_features] = df_pred[cat_features].fillna(value=df_train[cat_features].mode().iloc[0])
    df_train[num_features+cat_features].count
    
    all_good = (
    (np.prod(df_train[num_features+cat_features].shape)==df_train[num_features+cat_features].count().sum()) and 
    (np.prod(df_test[num_features+cat_features].shape) == df_test[num_features+cat_features].count().sum()) and 
    (np.prod(df_pred[num_features+cat_features].shape) == df_pred[num_features+cat_features].count().sum()))
    if (all_good):
        print('Missing values imputed successfully')
    else:
        print('There are still some missing values...')
    
def add_misDummy_mp_i1(df_train, df_test, df_pred, features):
    """This function creates new dummy columns for missing features.
    Example: add_misDummy_mp_i1(X_train, X_test, X_pred, ['Age'])"""
    # set df_pred to None if it does not exist
    for feature_name in features:
        misColName = 'mis'+feature_name
        df_train.loc[df_train[feature_name].isnull(), misColName]=1
        df_train.loc[df_train[feature_name].notnull(), misColName]=0
        df_test.loc[df_test[feature_name].isnull(), misColName]=1
        df_test.loc[df_test[feature_name].notnull(), misColName]=0
        if (df_pred is not None):
            df_pred.loc[df_pred[feature_name].isnull(), misColName]=1
            df_pred.loc[df_pred[feature_name].notnull(), misColName]=0
   

def discretize_mp_i1(df_train, df_test, df_pred, feature, ntiles, delete_feature=False):
    """This function divides a continuous feature into quantile groups.
    Example: discretize_mp_i1(X_train, X_test, X_pred, 'Age', 15)"""
    # set df_pred to None if it does not exist
    _,bin = pd.qcut(df_train[feature], ntiles, retbins = True, labels = False, duplicates = 'drop')
    df_train[feature+'Ntile'] = pd.cut(df_train[feature], labels=False, duplicates = 'drop', bins = bin ,include_lowest = True)
    df_test[feature+'Ntile'] = pd.cut(df_test[feature], labels=False, duplicates = 'drop', bins = bin ,include_lowest = True)
    if (df_pred is not None):
        df_pred[feature+'Ntile'] = pd.cut(df_pred[feature], labels=False, duplicates = 'drop', bins = bin ,include_lowest = True)
    if (delete_feature==True):
        df_train.drop(columns=[feature], inplace=True)
        df_test.drop(columns=[feature], inplace=True)
        df_pred.drop(columns=[feature], inplace=True)
    print('Discretized ',feature, ' into ', len(bin)-1, ' bins')


def log_transformer_mp_i1(df_train, df_test, df_pred, feature_subset=False, min_skew=3):
    """This function divides a continuous feature into quantile groups.
    Example: log_transformer_mp_i1(X_train, X_test, X_pred, feature_subset=num_cols)"""
    # set df_pred to None if it does not exist
    if (feature_subset==False):
        features_totransform = df_train.columns
    else:
        features_totransform = feature_subset.copy()
    skewed_vars = list(df_train.skew()[abs(df_train.skew())>min_skew].index)
    for col in list(set(skewed_vars)&set(features_totransform)):
        df_train[col] = np.log1p(df_train[col])
        df_test[col] = np.log1p(df_test[col])
        if (df_pred is not None):
            df_pred[col] = np.log1p(df_pred[col])
    print('Skewed columns log-transformed: ', list(set(skewed_vars)&set(features_totransform)))
    
    


In [174]:
with open('../input/amex-default-downsampled-01/amex_default_0.1sample.pickle', 'rb') as pickled_one:
    df = pickle.load(pickled_one)
display(df.head(),df.shape)

,customer_ID,S_2,P_2,D_39,B_1,B_2,R_1,S_3,D_41,B_3,D_42,D_43,D_44,B_4,D_45,B_5,R_2,D_46,D_47,D_48,D_49,B_6,B_7,B_8,D_50,D_51,B_9,R_3,D_52,P_3,B_10,D_53,S_5,B_11,S_6,D_54,R_4,S_7,B_12,S_8,D_55,D_56,B_13,R_5,D_58,S_9,B_14,D_59,D_60,D_61,B_15,S_11,D_62,D_63,D_64,D_65,B_16,B_17,B_18,B_19,D_66,B_20,D_68,S_12,R_6,S_13,B_21,D_69,B_22,D_70,D_71,D_72,S_15,B_23,D_73,P_4,D_74,D_75,D_76,B_24,R_7,D_77,B_25,B_26,D_78,D_79,R_8,R_9,S_16,D_80,R_10,R_11,B_27,D_81,D_82,S_17,R_12,B_28,R_13,D_83,R_14,R_15,D_84,R_16,B_29,B_30,S_18,D_86,D_87,R_17,R_18,D_88,B_31,S_19,R_19,B_32,S_20,R_20,R_21,B_33,D_89,R_22,R_23,D_91,D_92,D_93,D_94,R_24,R_25,D_96,S_22,S_23,S_24,S_25,S_26,D_102,D_103,D_104,D_105,D_106,D_107,B_36,B_37,R_26,R_27,B_38,D_108,D_109,D_110,D_111,B_39,D_112,B_40,S_27,D_113,D_114,D_115,D_116,D_117,D_118,D_119,D_120,D_121,D_122,D_123,D_124,D_125,D_126,D_127,D_128,D_129,B_41,B_42,D_130,D_131,D_132,D_133,R_28,D_134,D_135,D_136,D_137,D_138,D_139,D_140,D_141,D_142,D_143,D_144,D_145
0,00007889e4fcd2614b6cbe7f8f3d2e5c728eca32d9eb8a...,2017-05-30,0.937438,0.003936,0.003352,0.814304,0.002594,NaN,0.000545,0.003142,NaN,0.051788,0.006299,0.056203,0.294726,0.006012,0.004684,0.516281,0.463092,0.263574,NaN,0.048533,0.042849,0.006816,0.076369,0.004431,0.006775,0.005824,0.192826,0.577143,0.044728,NaN,0.003280,0.007864,1.000973,1.008453,0.000671,NaN,0.139550,0.003945,0.177336,0.169218,0.008859,0.009552,0.331046,NaN,0.005952,0.292875,0.002996,0.174586,0.005551,0.288992,0.081457,CO,O,0.009775,0.008577,NaN,0.645052,0.001056,1.0,0.001964,6.0,0.193990,0.000306,0.001171,0.000274,0.006808,0.005755,0.259391,0.015849,0.001570,0.507748,0.025547,NaN,0.001370,0.150138,0.137811,NaN,0.009684,0.004231,NaN,0.005078,0.003550,0.003895,0.000551,0.008775,NaN,0.003207,0.004396,0.004527,0.008564,0.006419,0.007386,NaN,0.000966,1.009854,0.058119,0.003344,0.007251,0.003969,0.008143,0.005722,0.009559,NaN,0.0,0.004709,0.001287,NaN,0.006435,0.008478,NaN,1,0.005740,0.005207,0.005587,0.000917,0.002636,0.001219,1.008942,0.000116,0.002600,0.001327,0.005468,0.006224,0.000029,0.003084,0.000054,0.005109,0.002801,0.301518,0.140075,0.082386,0.977059,0.001901,0.413177,1.000086,0.958687,0.661508,NaN,1.003331,0.006081,0.009483,NaN,1.005789,2.0,NaN,0.000268,NaN,NaN,NaN,1.009410,0.165166,NaN,0.001087,1.0,0.425165,0.0,4.0,0.418822,0.419459,0.0,0.547555,0.438120,0.003239,0.187171,0.006194,1.0,0.009093,1.005538,1.000016,0.003450,NaN,1.005729,0.001219,NaN,0.002394,0.004556,NaN,NaN,NaN,NaN,NaN,0.008103,0.006132,0.003267,NaN,0.008814,0.001211,0.001624
1,00013181a0c5fc8f1ea38cd2b90fe8ad2fa8cad9d9f13e...,2017-06-16,0.471242,0.001547,0.000233,0.816437,0.003437,NaN,0.005852,0.005877,NaN,0.208440,0.382538,0.313276,0.054264,0.002035,0.002264,0.951780,0.325470,0.740619,NaN,0.007790,1.047744,1.005298,0.076422,0.000206,0.005905,0.009041,0.148017,0.493458,0.039864,NaN,0.006881,0.003204,1.005171,1.001148,0.007656,NaN,0.010886,0.007330,0.448186,0.126476,0.004810,0.003998,0.619877,NaN,0.003674,0.375356,0.001085,0.599261,0.004723,0.285493,0.017829,CO,O,0.001949,0.005428,NaN,0.532836,0.001264,NaN,0.002074,5.0,0.184956,0.004232,0.004607,0.000368,0.002620,0.003322,1.008265,0.034615,0.001517,0.500245,0.995367,NaN,0.001463,0.365847,0.338917,NaN,0.007694,0.004741,NaN,0.004701,0.009776,0.004254,0.001066,0.003238,0.171472,0.003671,0.409818,0.001820,0.001104,0.008046,0.001091,NaN,0.001583,1.004534,0.211125,0.002501,0.007636,0.009532,0.004129,0.001284,0.008175,NaN,0.0,0.003752,0.004509,NaN,0.001339,0.008494,NaN,1,0.005238,0.008327,0.002736,0.007807,0.005138,0.009324,1.006314,0.005657,0.006198,0.006972,0.007000,0.001464,0.001995,0.004677,0.009206,0.008958,0.008415,0.302657,0.134319,0.086714,0.972725,0.000080,0.003580,0.008142,0.003662,NaN,NaN,0.008267,0.000910,0.002263,0.081330,0.015774,1.0,NaN,0.005904,NaN,NaN,NaN,1.002032,1.113697,NaN,0.004361,1.0,0.044666,0.0,4.0,0.045775,0.044385,0.0,0.755079,0.291698,0.009170,0.455286,0.006972,1.0,0.008171,1.004957,1.009854,0.000533,NaN,1.002896,0.006603,NaN,0.007623,0.006064,NaN,NaN,NaN,NaN,NaN

(54849, 190)

In [175]:
df_labels = pd.read_csv('../input/amex-default-prediction/train_labels.csv')
display(len(df_labels))
df = pd.merge(df, df_labels, on='customer_ID', how='left')
display(df.shape, df.head())
df.drop(columns=['customer_ID', 'S_2'], inplace=True)

458913

(54849, 191)

,customer_ID,S_2,P_2,D_39,B_1,B_2,R_1,S_3,D_41,B_3,D_42,D_43,D_44,B_4,D_45,B_5,R_2,D_46,D_47,D_48,D_49,B_6,B_7,B_8,D_50,D_51,B_9,R_3,D_52,P_3,B_10,D_53,S_5,B_11,S_6,D_54,R_4,S_7,B_12,S_8,D_55,D_56,B_13,R_5,D_58,S_9,B_14,D_59,D_60,D_61,B_15,S_11,D_62,D_63,D_64,D_65,B_16,B_17,B_18,B_19,D_66,B_20,D_68,S_12,R_6,S_13,B_21,D_69,B_22,D_70,D_71,D_72,S_15,B_23,D_73,P_4,D_74,D_75,D_76,B_24,R_7,D_77,B_25,B_26,D_78,D_79,R_8,R_9,S_16,D_80,R_10,R_11,B_27,D_81,D_82,S_17,R_12,B_28,R_13,D_83,R_14,R_15,D_84,R_16,B_29,B_30,S_18,D_86,D_87,R_17,R_18,D_88,B_31,S_19,R_19,B_32,S_20,R_20,R_21,B_33,D_89,R_22,R_23,D_91,D_92,D_93,D_94,R_24,R_25,D_96,S_22,S_23,S_24,S_25,S_26,D_102,D_103,D_104,D_105,D_106,D_107,B_36,B_37,R_26,R_27,B_38,D_108,D_109,D_110,D_111,B_39,D_112,B_40,S_27,D_113,D_114,D_115,D_116,D_117,D_118,D_119,D_120,D_121,D_122,D_123,D_124,D_125,D_126,D_127,D_128,D_129,B_41,B_42,D_130,D_131,D_132,D_133,R_28,D_134,D_135,D_136,D_137,D_138,D_139,D_140,D_141,D_142,D_143,D_144,D_145,target
0,00007889e4fcd2614b6cbe7f8f3d2e5c728eca32d9eb8a...,2017-05-30,0.937438,0.003936,0.003352,0.814304,0.002594,NaN,0.000545,0.003142,NaN,0.051788,0.006299,0.056203,0.294726,0.006012,0.004684,0.516281,0.463092,0.263574,NaN,0.048533,0.042849,0.006816,0.076369,0.004431,0.006775,0.005824,0.192826,0.577143,0.044728,NaN,0.003280,0.007864,1.000973,1.008453,0.000671,NaN,0.139550,0.003945,0.177336,0.169218,0.008859,0.009552,0.331046,NaN,0.005952,0.292875,0.002996,0.174586,0.005551,0.288992,0.081457,CO,O,0.009775,0.008577,NaN,0.645052,0.001056,1.0,0.001964,6.0,0.193990,0.000306,0.001171,0.000274,0.006808,0.005755,0.259391,0.015849,0.001570,0.507748,0.025547,NaN,0.001370,0.150138,0.137811,NaN,0.009684,0.004231,NaN,0.005078,0.003550,0.003895,0.000551,0.008775,NaN,0.003207,0.004396,0.004527,0.008564,0.006419,0.007386,NaN,0.000966,1.009854,0.058119,0.003344,0.007251,0.003969,0.008143,0.005722,0.009559,NaN,0.0,0.004709,0.001287,NaN,0.006435,0.008478,NaN,1,0.005740,0.005207,0.005587,0.000917,0.002636,0.001219,1.008942,0.000116,0.002600,0.001327,0.005468,0.006224,0.000029,0.003084,0.000054,0.005109,0.002801,0.301518,0.140075,0.082386,0.977059,0.001901,0.413177,1.000086,0.958687,0.661508,NaN,1.003331,0.006081,0.009483,NaN,1.005789,2.0,NaN,0.000268,NaN,NaN,NaN,1.009410,0.165166,NaN,0.001087,1.0,0.425165,0.0,4.0,0.418822,0.419459,0.0,0.547555,0.438120,0.003239,0.187171,0.006194,1.0,0.009093,1.005538,1.000016,0.003450,NaN,1.005729,0.001219,NaN,0.002394,0.004556,NaN,NaN,NaN,NaN,NaN,0.008103,0.006132,0.003267,NaN,0.008814,0.001211,0.001624,0
1,00013181a0c5fc8f1ea38cd2b90fe8ad2fa8cad9d9f13e...,2017-06-16,0.471242,0.001547,0.000233,0.816437,0.003437,NaN,0.005852,0.005877,NaN,0.208440,0.382538,0.313276,0.054264,0.002035,0.002264,0.951780,0.325470,0.740619,NaN,0.007790,1.047744,1.005298,0.076422,0.000206,0.005905,0.009041,0.148017,0.493458,0.039864,NaN,0.006881,0.003204,1.005171,1.001148,0.007656,NaN,0.010886,0.007330,0.448186,0.126476,0.004810,0.003998,0.619877,NaN,0.003674,0.375356,0.001085,0.599261,0.004723,0.285493,0.017829,CO,O,0.001949,0.005428,NaN,0.532836,0.001264,NaN,0.002074,5.0,0.184956,0.004232,0.004607,0.000368,0.002620,0.003322,1.008265,0.034615,0.001517,0.500245,0.995367,NaN,0.001463,0.365847,0.338917,NaN,0.007694,0.004741,NaN,0.004701,0.009776,0.004254,0.001066,0.003238,0.171472,0.003671,0.409818,0.001820,0.001104,0.008046,0.001091,NaN,0.001583,1.004534,0.211125,0.002501,0.007636,0.009532,0.004129,0.001284,0.008175,NaN,0.0,0.003752,0.004509,NaN,0.001339,0.008494,NaN,1,0.005238,0.008327,0.002736,0.007807,0.005138,0.009324,1.006314,0.005657,0.006198,0.006972,0.007000,0.001464,0.001995,0.004677,0.009206,0.008958,0.008415,0.302657,0.134319,0.086714,0.972725,0.000080,0.003580,0.008142,0.003662,NaN,NaN,0.008267,0.000910,0.002263,0.081330,0.015774,1.0,NaN,0.005904,NaN,NaN,NaN,1.002032,1.113697,NaN,0.004361,1.0,0.044666,0.0,4.0,0.045775,0.044385,0.0,0.755079,0.291698,0.009170,0.455286,0.006972,1.0,0.008171,1.004957,1.009854,0.000533,NaN,1.002896,0.006603,NaN,0.007623,0.006064,NaN,NaN,Na

In [176]:
miss_c = df.count()
miss_feat = miss_c[miss_c<40000]
df = df[list(set(df.columns)-set(miss_feat.index))]
display(df.shape)
df[['D_114','D_120','D_68','B_30','D_117','D_116','B_38']] = \
df[['D_114','D_120','D_68','B_30','D_117','D_116','B_38']].apply(pd.to_numeric)
df['D_64'] = df['D_64'].astype('string')
display(df.dtypes)

(54849, 157)

D_47      float64
B_5       float64
R_24      float64
D_65      float64
B_19      float64
D_119     float64
S_5       float64
B_14      float64
S_26      float64
D_91      float64
B_9       float64
R_16      float64
D_81      float64
D_133     float64
P_4       float64
B_37      float64
S_12      float64
D_89      float64
D_61      float64
D_122     float64
S_25      float64
R_8       float64
R_20      float64
D_92      float64
B_20      float64
D_144     float64
D_70      float64
B_27      float64
D_48      float64
target      int64
B_40      float64
B_22      float64
S_13      float64
B_25      float64
D_114     float64
B_33      float64
B_4       float64
S_15      float64
D_74      float64
S_23      float64
S_11      float64
B_7       float64
D_143     float64
S_6       float64
D_93      float64
D_131     float64
D_121     float64
R_25      float64
B_3       float64
D_120     float64
D_39      float64
R_14      float64
D_113     float64
D_115     float64
D_140     float64
R_15      

In [177]:
### sample split ###

test_size = 0.1
df.reset_index(inplace=True, drop=True)
test_index = random.sample(list(df.index), int(test_size*df.shape[0]))
train = df.iloc[list(set(df.index)-set(test_index))]
test = df.iloc[test_index]
display(train.shape, test.shape, train.head(3), test.head(3))
display(train.dtypes, test.dtypes)


(49365, 157)

(5484, 157)

,D_47,B_5,R_24,D_65,B_19,D_119,S_5,B_14,S_26,D_91,B_9,R_16,D_81,D_133,P_4,B_37,S_12,D_89,D_61,D_122,S_25,R_8,R_20,D_92,B_20,D_144,D_70,B_27,D_48,target,B_40,B_22,S_13,B_25,D_114,B_33,B_4,S_15,D_74,S_23,S_11,B_7,D_143,S_6,D_93,D_131,D_121,R_25,B_3,D_120,D_39,R_14,D_113,D_115,D_140,R_15,D_55,D_118,D_69,B_12,P_2,D_44,S_24,R_28,S_27,B_24,D_80,D_68,B_6,B_11,B_30,R_1,S_8,R_22,D_124,S_20,B_26,D_129,D_125,D_127,D_86,B_8,R_13,D_128,R_4,D_62,D_59,S_16,D_103,D_83,D_60,D_71,S_18,B_16,B_32,D_54,B_10,R_6,D_84,B_31,B_21,R_5,R_2,R_12,R_23,R_19,D_63,B_23,R_10,R_17,R_21,D_102,B_13,B_28,B_2,D_45,D_130,D_139,R_27,D_141,B_18,D_117,S_22,D_107,S_3,D_51,D_52,B_36,D_41,B_41,D_145,D_116,S_19,D_72,R_3,D_79,D_109,D_78,D_112,R_7,D_96,D_123,D_75,B_38,D_126,D_46,S_7,P_3,R_11,D_58,D_64,D_94,S_17,D_104,B_1,B_15,R_18
0,0.463092,0.006012,0.000054,0.009775,0.001056,0.419459,0.003280,0.005952,0.001901,0.005468,0.006775,0.009559,0.007386,0.002394,0.001370,0.009483,0.193990,0.000116,0.174586,0.438120,0.977059,0.008775,0.002636,0.006224,0.001964,0.001211,0.259391,0.006419,0.263574,0,0.165166,0.005755,0.001171,0.005078,1.0,1.008942,0.056203,0.507748,0.150138,0.140075,0.288992,0.042849,0.008814,1.000973,0.000029,0.001219,0.547555,0.005109,0.003142,0.0,0.003936,0.003969,0.001087,0.425165,0.006132,0.008143,0.177336,0.418822,0.006808,0.139550,0.937438,0.006299,0.082386,0.004556,NaN,0.009684,0.004396,6.0,0.048533,0.007864,0.0,0.002594,0.003945,0.002600,0.187171,0.000917,0.003550,1.000016,0.006194,0.009093,0.001287,0.006816,0.003344,1.005538,0.000671,0.081457,0.292875,0.003207,1.000086,0.007251,0.002996,0.015849,0.004709,0.008577,0.005587,1.008453,0.044728,0.000306,0.005722,1,0.000274,0.009552,0.004684,1.009854,0.001327,0.005207,CO,0.025547,0.004527,0.006435,0.001219,0.413177,0.008859,0.058119,0.814304,0.294726,1.005729,0.008103,1.005789,0.003267,0.645052,4.0,0.301518,1.003331,NaN,0.004431,0.192826,0.006081,0.000545,0.003450,0.001624,0.0,0.005740,0.001570,0.005824,0.000551,0.000268,0.003895,1.009410,0.004231,0.002801,0.003239,0.137811,2.0,1.0,0.516281,NaN,0.577143,0.008564,0.331046,O,0.003084,0.000966,0.958687,0.003352,0.005551,0.008478
1,0.325470,0.002035,0.009206,0.001949,0.001264,0.044385,0.006881,0.003674,0.000080,0.007000,0.005905,0.008175,0.001091,0.007623,0.001463,0.002263,0.184956,0.005657,0.599261,0.291698,0.972725,0.003238,0.005138,0.001464,0.002074,0.007118,1.008265,0.008046,0.740619,1,1.113697,0.003322,0.004607,0.004701,1.0,1.006314,0.313276,0.500245,0.365847,0.134319,0.285493,1.047744,0.006323,1.005171,0.001995,0.006603,0.755079,0.008958,0.005877,0.0,0.001547,0.009532,0.004361,0.044666,0.005849,0.004129,0.448186,0.045775,0.002620,0.010886,0.471242,0.382538,0.086714,0.006064,NaN,0.007694,0.409818,5.0,0.007790,0.003204,0.0,0.003437,0.007330,0.006198,0.455286,0.007807,0.009776,1.009854,0.006972,0.008171,0.004509,1.005298,0.002501,1.004957,0.007656,0.017829,0.375356,0.003671,0.008142,0.007636,0.001085,0.034615,0.003752,0.005428,0.002736,1.001148,0.039864,0.004232,0.001284,1,0.000368,0.003998,0.002264,1.004534,0.006972,0.008327,CO,0.995367,0.001820,0.001339,0.009324,0.003580,0.004810,0.211125,0.816437,0.054264,1.002896,0.009825,0.015774,0.003350,0.532836,4.0,0.302657,0.008267,NaN,0.000206,0.148017,0.000910,0.005852,0.000533,0.007420,0.0,0.005238,0.001517,0.009041,0.001066,0.005904,0.004254,1.002032,0.004741,0.008415,0.009170,0.338917,1.0,1.0,0.951780,NaN,0.493458,0.001104,0.619877,O,0.004677,0.001583,0.003662,0.000233,0.004723,0.008494
3,0.419005,0.006022,0.009982,0.002262,0.004585,0.345898,0.002898,0.002678,0.009972,0.005616,0.009491,0.000597,0.009827,0.004805,0.001955,0.005884,0.188458,0.006844,0.018364,0.723254,0.969548,0.000318,0.001946,0.005936,0.000540,0.002958,0.001627,0.007878,0.016205,0,0.064525,0.007346,0.008547,0.008383,1.0,1.009686,0.000623,0.204400,0.001150,0.134354,0.444328,0.030058,0.000389,1.007817,0.005590,0.007497,0.718976,0.007121,0.005782,0.0,0.007491,0.008848,0.005334,0.354861,0.009969,0.009341,0.023306,0.348603,0.006651,0.012557,1.004737,0.004

,D_47,B_5,R_24,D_65,B_19,D_119,S_5,B_14,S_26,D_91,B_9,R_16,D_81,D_133,P_4,B_37,S_12,D_89,D_61,D_122,S_25,R_8,R_20,D_92,B_20,D_144,D_70,B_27,D_48,target,B_40,B_22,S_13,B_25,D_114,B_33,B_4,S_15,D_74,S_23,S_11,B_7,D_143,S_6,D_93,D_131,D_121,R_25,B_3,D_120,D_39,R_14,D_113,D_115,D_140,R_15,D_55,D_118,D_69,B_12,P_2,D_44,S_24,R_28,S_27,B_24,D_80,D_68,B_6,B_11,B_30,R_1,S_8,R_22,D_124,S_20,B_26,D_129,D_125,D_127,D_86,B_8,R_13,D_128,R_4,D_62,D_59,S_16,D_103,D_83,D_60,D_71,S_18,B_16,B_32,D_54,B_10,R_6,D_84,B_31,B_21,R_5,R_2,R_12,R_23,R_19,D_63,B_23,R_10,R_17,R_21,D_102,B_13,B_28,B_2,D_45,D_130,D_139,R_27,D_141,B_18,D_117,S_22,D_107,S_3,D_51,D_52,B_36,D_41,B_41,D_145,D_116,S_19,D_72,R_3,D_79,D_109,D_78,D_112,R_7,D_96,D_123,D_75,B_38,D_126,D_46,S_7,P_3,R_11,D_58,D_64,D_94,S_17,D_104,B_1,B_15,R_18
45810,-0.023208,0.006786,0.003466,0.008527,0.001508,NaN,0.001776,0.004528,0.003382,NaN,0.009095,0.002627,NaN,NaN,0.961960,0.009493,0.187637,NaN,0.277665,NaN,0.975758,0.007385,0.000985,0.009452,0.007116,NaN,NaN,0.003043,NaN,1,0.008105,0.005367,0.007226,0.003385,NaN,1.007688,0.002959,0.509830,0.004201,0.131963,0.288504,0.000445,NaN,1.008822,0.000508,NaN,NaN,0.000818,0.009632,NaN,0.007341,0.006255,NaN,NaN,NaN,0.004117,NaN,NaN,NaN,0.006373,NaN,NaN,0.193343,0.002906,NaN,0.007365,0.004658,NaN,0.181780,0.008009,0.0,0.005394,0.000731,0.001813,NaN,0.006377,0.006280,NaN,NaN,0.006197,0.005165,1.004060,0.003365,NaN,0.005520,NaN,NaN,0.006663,NaN,NaN,1.004641,0.016570,0.004361,0.006806,0.008162,1.007515,0.245815,0.009645,NaN,1,0.007994,0.000439,0.008274,1.006727,0.005689,0.009188,CO,0.004168,0.003507,0.005678,0.005809,NaN,NaN,0.009287,0.817170,0.003187,NaN,NaN,1.007891,NaN,1.009506,NaN,0.347436,NaN,NaN,0.007113,NaN,0.000436,0.000992,0.004802,NaN,NaN,0.005245,NaN,0.005611,NaN,0.000364,NaN,1.004163,0.008254,0.003215,NaN,0.009233,1.0,NaN,NaN,NaN,NaN,0.007088,0.003364,<NA>,0.008730,0.008270,NaN,0.001520,0.000055,0.007110
30971,0.882235,0.001231,0.000635,0.000694,0.000915,0.643212,0.000616,0.005195,0.006847,0.008704,0.000846,0.006711,0.008380,0.003372,0.006845,0.002553,0.194232,0.008932,0.045547,0.286309,0.978525,0.006119,0.009416,0.002561,0.004444,0.005066,0.007193,0.009455,0.048362,0,0.035690,0.007811,0.000865,0.004076,1.0,1.008110,0.031986,0.506095,0.006426,0.139927,0.288057,0.034703,0.005483,1.009783,0.007126,0.003805,0.686830,0.005400,0.000204,0.0,0.006754,0.002435,0.004639,0.421795,0.005603,0.005255,0.049583,0.651547,0.005919,0.030137,0.926444,0.009402,0.005098,0.005437,NaN,0.003375,0.004750,6.0,0.190054,0.007138,0.0,0.002586,0.009827,0.005004,0.227330,0.009577,0.005929,0.008317,0.006107,0.006832,0.004245,0.001808,0.005766,1.007622,0.005447,0.195839,0.069902,0.009814,0.000064,0.006861,0.000496,0.010825,0.008400,0.005234,0.002425,1.003802,0.300936,0.006934,0.005915,1,0.009009,0.000765,0.009786,1.004412,0.000543,0.003454,CR,0.020000,0.006403,0.008199,0.003198,0.006170,0.009382,0.022894,0.815761,0.687521,0.008473,0.004954,1.007397,0.000990,1.001000,4.0,0.042244,0.001245,NaN,0.007328,0.213015,0.003722,0.002623,0.008965,0.004058,0.0,0.008640,0.008713,0.104705,0.000325,0.000320,0.001913,1.003816,0.008051,0.006175,0.002976,0.002982,1.0,1.0,0.439917,NaN,0.642611,0.502459,0.007237,O,0.009495,0.002342,0.006740,0.004330,0.003991,0.009050
2109,0.203856,0.002919,0.002427,0.002974,0.002234,0.097924,0.004441,0.005810,0.005500,0.008004,0.006830,0.008449,0.002752,0.008532,0.001927,0.001703,0.192424,0.006783,0.136817,0.434564,0.978624,0.008124,0.002965,0.006519,0.003894,0.001081,0.006288,0.009161,0.169444,0,0.014813,0.003039,0.006782,0.002961,1.0,1.000559,0.007749,0.501207,0.008978,0.140328,0.286253,0.017963,0.008063,0.007345,0.004199,0.005153,0.284347,0.009521,0.007460,0.0,0.004988,0.005544,0.407032,0.098840,0.007502,0.001671,0.080897,0.093032,0.007498,0.022164,0.859788,0.006692,0.077500,0.006197,NaN,0.005049,0.004947,4.0,0.342880,0.001158,0.0,0.007896,0.007632,0.005410,0.229406,0.002517,0.004734,1.008839,0.001194,0.004114,0.002408,1.003322,0.004736,1.005547,0.008878,0.457775,

D_47      float64
B_5       float64
R_24      float64
D_65      float64
B_19      float64
D_119     float64
S_5       float64
B_14      float64
S_26      float64
D_91      float64
B_9       float64
R_16      float64
D_81      float64
D_133     float64
P_4       float64
B_37      float64
S_12      float64
D_89      float64
D_61      float64
D_122     float64
S_25      float64
R_8       float64
R_20      float64
D_92      float64
B_20      float64
D_144     float64
D_70      float64
B_27      float64
D_48      float64
target      int64
B_40      float64
B_22      float64
S_13      float64
B_25      float64
D_114     float64
B_33      float64
B_4       float64
S_15      float64
D_74      float64
S_23      float64
S_11      float64
B_7       float64
D_143     float64
S_6       float64
D_93      float64
D_131     float64
D_121     float64
R_25      float64
B_3       float64
D_120     float64
D_39      float64
R_14      float64
D_113     float64
D_115     float64
D_140     float64
R_15      

D_47      float64
B_5       float64
R_24      float64
D_65      float64
B_19      float64
D_119     float64
S_5       float64
B_14      float64
S_26      float64
D_91      float64
B_9       float64
R_16      float64
D_81      float64
D_133     float64
P_4       float64
B_37      float64
S_12      float64
D_89      float64
D_61      float64
D_122     float64
S_25      float64
R_8       float64
R_20      float64
D_92      float64
B_20      float64
D_144     float64
D_70      float64
B_27      float64
D_48      float64
target      int64
B_40      float64
B_22      float64
S_13      float64
B_25      float64
D_114     float64
B_33      float64
B_4       float64
S_15      float64
D_74      float64
S_23      float64
S_11      float64
B_7       float64
D_143     float64
S_6       float64
D_93      float64
D_131     float64
D_121     float64
R_25      float64
B_3       float64
D_120     float64
D_39      float64
R_14      float64
D_113     float64
D_115     float64
D_140     float64
R_15      

In [178]:
cat_feat = ['B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_68']
num_feat = list(set(train.columns)-set(cat_feat)-set(['target']))
display(train[cat_feat], [train[col].value_counts() for col in train.columns if col in cat_feat])

,B_30,B_38,D_114,D_116,D_117,D_120,D_126,D_63,D_64,D_68
0,0.0,2.0,1.0,0.0,4.0,0.0,1.0,CO,O,6.0
1,0.0,1.0,1.0,0.0,4.0,0.0,1.0,CO,O,5.0
3,0.0,1.0,1.0,0.0,4.0,0.0,1.0,CO,O,6.0
4,1.0,6.0,0.0,0.0,-1.0,0.0,1.0,CO,U,3.0
5,0.0,2.0,1.0,0.0,-1.0,0.0,1.0,CL,R,6.0
...,...,...,...,...,...,...,...,...,...,...
54844,0.0,2.0,1.0,0.0,4.0,1.0,1.0,CR,O,5.0
54845,0.0,3.0,1.0,0.0,-1.0,0.0,1.0,CO,U,5.0
54846,0.0,2.0,1.0,0.0,5.0,0.0,1.0,CL,O,5.0
54847,0.0,1.0,NaN,NaN,NaN,NaN,NaN,CO,<NA>,NaN


[1.0    29495
 0.0    18312
 Name: D_114, dtype: int64,
 0.0    42177
 1.0     5630
 Name: D_120, dtype: int64,
 6.0    24846
 5.0    10718
 3.0     4316
 4.0     4239
 2.0     1959
 1.0     1235
 0.0      151
 Name: D_68, dtype: int64,
 0.0    42086
 1.0     6804
 2.0      454
 Name: B_30, dtype: int64,
 CO    36866
 CR     8241
 CL     3918
 XZ      194
 XM       83
 XL       63
 Name: D_63, dtype: int64,
 -1.0    12988
  3.0    10493
  4.0    10220
  2.0     5877
  5.0     3988
  6.0     3111
  1.0     1130
 Name: D_117, dtype: int64,
 0.0    47742
 1.0       65
 Name: D_116, dtype: int64,
 2.0    17274
 3.0    11357
 1.0    10332
 5.0     3972
 4.0     2595
 7.0     2269
 6.0     1545
 Name: B_38, dtype: int64,
  1.0    37966
  0.0     8116
 -1.0     2265
 Name: D_126, dtype: int64,
 O     25849
 U     13706
 R      7553
 -1      353
 Name: D_64, dtype: Int64]

In [179]:
for col in cat_feat:
    if train[col].dtype in ['string', 'O', 'str']:
        print('str', col)
    else:
        print('num', col)
        

num B_30
num B_38
num D_114
num D_116
num D_117
num D_120
num D_126
str D_63
str D_64
num D_68


In [180]:
### fill missing values ###

for col in cat_feat:
    if train[col].dtype in ['string', 'O', 'str']:
        train[col].fillna(value='M', inplace=True)
        test[col].fillna(value='M', inplace=True)
    else:
        train[col].fillna(value=-1000, inplace=True)
        test[col].fillna(value=-1000, inplace=True)

for col in num_feat:
    train[col].fillna(train[col].median(), inplace=True)
    test[col].fillna(train[col].median(), inplace=True)
    
#display(train.count(), test.count())
#display(train.count(), test.count())

In [181]:
train.dtypes

D_47      float64
B_5       float64
R_24      float64
D_65      float64
B_19      float64
D_119     float64
S_5       float64
B_14      float64
S_26      float64
D_91      float64
B_9       float64
R_16      float64
D_81      float64
D_133     float64
P_4       float64
B_37      float64
S_12      float64
D_89      float64
D_61      float64
D_122     float64
S_25      float64
R_8       float64
R_20      float64
D_92      float64
B_20      float64
D_144     float64
D_70      float64
B_27      float64
D_48      float64
target      int64
B_40      float64
B_22      float64
S_13      float64
B_25      float64
D_114     float64
B_33      float64
B_4       float64
S_15      float64
D_74      float64
S_23      float64
S_11      float64
B_7       float64
D_143     float64
S_6       float64
D_93      float64
D_131     float64
D_121     float64
R_25      float64
B_3       float64
D_120     float64
D_39      float64
R_14      float64
D_113     float64
D_115     float64
D_140     float64
R_15      

In [182]:
### OHE ###

X_train = train.copy()
y_train = X_train.pop('target')
X_test = test.copy()
y_test = X_test.pop('target')
display(X_test.head())
display(X_train.nunique())

### Do OHE for some features ###

# this code uses passthrough from 
# https://stackoverflow.com/questions/54160370/how-to-use-sklearn-column-transformer

feature_transformer = ColumnTransformer([
    ("cat", OneHotEncoder(sparse = False, handle_unknown="ignore"), cat_feat)],
    remainder="passthrough")

print('Number of features before transaformation: ', X_train.shape)
X_train = pd.DataFrame(feature_transformer.fit_transform(X_train), columns=feature_transformer.get_feature_names_out())
X_test = pd.DataFrame(feature_transformer.transform(X_test), columns=feature_transformer.get_feature_names_out())

print('time to do feature proprocessing: ', time.time()-time1)

print('Number of features after transaformation: ', X_train.shape)

,D_47,B_5,R_24,D_65,B_19,D_119,S_5,B_14,S_26,D_91,B_9,R_16,D_81,D_133,P_4,B_37,S_12,D_89,D_61,D_122,S_25,R_8,R_20,D_92,B_20,D_144,D_70,B_27,D_48,B_40,B_22,S_13,B_25,D_114,B_33,B_4,S_15,D_74,S_23,S_11,B_7,D_143,S_6,D_93,D_131,D_121,R_25,B_3,D_120,D_39,R_14,D_113,D_115,D_140,R_15,D_55,D_118,D_69,B_12,P_2,D_44,S_24,R_28,S_27,B_24,D_80,D_68,B_6,B_11,B_30,R_1,S_8,R_22,D_124,S_20,B_26,D_129,D_125,D_127,D_86,B_8,R_13,D_128,R_4,D_62,D_59,S_16,D_103,D_83,D_60,D_71,S_18,B_16,B_32,D_54,B_10,R_6,D_84,B_31,B_21,R_5,R_2,R_12,R_23,R_19,D_63,B_23,R_10,R_17,R_21,D_102,B_13,B_28,B_2,D_45,D_130,D_139,R_27,D_141,B_18,D_117,S_22,D_107,S_3,D_51,D_52,B_36,D_41,B_41,D_145,D_116,S_19,D_72,R_3,D_79,D_109,D_78,D_112,R_7,D_96,D_123,D_75,B_38,D_126,D_46,S_7,P_3,R_11,D_58,D_64,D_94,S_17,D_104,B_1,B_15,R_18
45810,-0.023208,0.006786,0.003466,0.008527,0.001508,0.215585,0.001776,0.004528,0.003382,0.005540,0.009095,0.002627,0.005194,0.005458,0.961960,0.009493,0.187637,0.005054,0.277665,0.429217,0.975758,0.007385,0.000985,0.009452,0.007116,0.005513,0.006781,0.003043,0.287675,0.008105,0.005367,0.007226,0.003385,-1000.0,1.007688,0.002959,0.509830,0.004201,0.131963,0.288504,0.000445,0.006120,1.008822,0.000508,0.005563,0.595825,0.000818,0.009632,-1000.0,0.007341,0.006255,0.008699,0.198263,0.005117,0.004117,0.183962,0.222743,0.005217,0.006373,0.692421,0.007659,0.193343,0.002906,0.292374,0.007365,0.004658,-1000.0,0.181780,0.008009,0.0,0.005394,0.000731,0.001813,0.273516,0.006377,0.006280,0.008811,0.005391,0.006197,0.005165,1.004060,0.003365,1.000395,0.005520,0.093395,0.379306,0.006663,0.009310,0.005138,1.004641,0.016570,0.004361,0.006806,0.008162,1.007515,0.245815,0.009645,0.005233,1,0.007994,0.000439,0.008274,1.006727,0.005689,0.009188,CO,0.004168,0.003507,0.005678,0.005809,0.008929,0.029184,0.009287,0.817170,0.003187,0.006228,0.006082,1.007891,0.006092,1.009506,-1000.0,0.347436,0.009325,0.163856,0.007113,0.143960,0.000436,0.000992,0.004802,0.006074,-1000.0,0.005245,0.005259,0.005611,0.005488,0.000364,0.005393,1.004163,0.008254,0.003215,0.005251,0.009233,1.0,-1000.0,0.459904,0.139736,0.617932,0.007088,0.003364,M,0.008730,0.008270,0.009303,0.001520,0.000055,0.007110
30971,0.882235,0.001231,0.000635,0.000694,0.000915,0.643212,0.000616,0.005195,0.006847,0.008704,0.000846,0.006711,0.008380,0.003372,0.006845,0.002553,0.194232,0.008932,0.045547,0.286309,0.978525,0.006119,0.009416,0.002561,0.004444,0.005066,0.007193,0.009455,0.048362,0.035690,0.007811,0.000865,0.004076,1.0,1.008110,0.031986,0.506095,0.006426,0.139927,0.288057,0.034703,0.005483,1.009783,0.007126,0.003805,0.686830,0.005400,0.000204,0.0,0.006754,0.002435,0.004639,0.421795,0.005603,0.005255,0.049583,0.651547,0.005919,0.030137,0.926444,0.009402,0.005098,0.005437,0.292374,0.003375,0.004750,6.0,0.190054,0.007138,0.0,0.002586,0.009827,0.005004,0.227330,0.009577,0.005929,0.008317,0.006107,0.006832,0.004245,0.001808,0.005766,1.007622,0.005447,0.195839,0.069902,0.009814,0.000064,0.006861,0.000496,0.010825,0.008400,0.005234,0.002425,1.003802,0.300936,0.006934,0.005915,1,0.009009,0.000765,0.009786,1.004412,0.000543,0.003454,CR,0.020000,0.006403,0.008199,0.003198,0.006170,0.009382,0.022894,0.815761,0.687521,0.008473,0.004954,1.007397,0.000990,1.001000,4.0,0.042244,0.001245,0.163856,0.007328,0.213015,0.003722,0.002623,0.008965,0.004058,0.0,0.008640,0.008713,0.104705,0.000325,0.000320,0.001913,1.003816,0.008051,0.006175,0.002976,0.002982,1.0,1.0,0.439917,0.139736,0.642611,0.502459,0.007237,O,0.009495,0.002342,0.006740,0.004330,0.003991,0.009050
2109,0.203856,0.002919,0.002427,0.002974,0.002234,0.097924,0.004441,0.005810,0.005500,0.008004,0.006830,0.008449,0.002752,0.008532,0.001927,0.001703,0.192424,0.006783,0.136817,0.434564,0.978624,0.008124,0.002965,0.006519,0.003894,0.001081,0.006288,0.009161,0.169444,0.014813,0.003039,0.006782,0.002961,1.0,1.000559,0.007749,0.501207,0.008978,0.140328,0.286253,0.017963,0.008063,0.007345,0.004199,0.005153,0.284347,0.009521,0.007460,0.0,0.004988,0.005544,0.407032,0.098840,0.007502,0.0016

D_47     49365
B_5      49365
R_24     49365
D_65     49365
B_19     49345
D_119    47807
S_5      49365
B_14     49365
S_26     49361
D_91     47987
B_9      49365
R_16     49365
D_81     49153
D_133    49013
P_4      49365
B_37     49365
S_12     49365
D_89     49121
D_61     44017
D_122    47807
S_25     49255
R_8      49365
R_20     49365
D_92     49365
B_20     49345
D_144    49033
D_70     48529
B_27     49345
D_48     42933
B_40     49365
B_22     49345
S_13     49365
B_25     49299
D_114        3
B_33     49345
B_4      49365
S_15     49365
D_74     49167
S_23     49361
S_11     49365
B_7      49365
D_143    48465
S_6      49365
D_93     49365
D_131    48465
D_121    47807
R_25     49365
B_3      49345
D_120        3
D_39     49365
R_14     49365
D_113    47807
D_115    47807
D_140    49035
R_15     49365
D_55     47701
D_118    47807
D_69     47659
B_12     49365
P_2      48947
D_44     46937
S_24     49205
R_28     49365
S_27     36905
B_24     49365
D_80     49167
D_68      

Number of features before transaformation:  (49365, 156)
time to do feature proprocessing:  4589.863786697388
Number of features after transaformation:  (49365, 198)


In [184]:
str_feat = [col for col in train.columns if train[col].dtype=='object']
str_feat

['D_63']

In [185]:
train[str_feat].head(10)

,D_63
0,CO
1,CO
3,CO
4,CO
5,CL
6,CO
7,CO
8,CO
9,CL
10,CO


In [186]:
train.dtypes

D_47      float64
B_5       float64
R_24      float64
D_65      float64
B_19      float64
D_119     float64
S_5       float64
B_14      float64
S_26      float64
D_91      float64
B_9       float64
R_16      float64
D_81      float64
D_133     float64
P_4       float64
B_37      float64
S_12      float64
D_89      float64
D_61      float64
D_122     float64
S_25      float64
R_8       float64
R_20      float64
D_92      float64
B_20      float64
D_144     float64
D_70      float64
B_27      float64
D_48      float64
target      int64
B_40      float64
B_22      float64
S_13      float64
B_25      float64
D_114     float64
B_33      float64
B_4       float64
S_15      float64
D_74      float64
S_23      float64
S_11      float64
B_7       float64
D_143     float64
S_6       float64
D_93      float64
D_131     float64
D_121     float64
R_25      float64
B_3       float64
D_120     float64
D_39      float64
R_14      float64
D_113     float64
D_115     float64
D_140     float64
R_15      

In [187]:
train.describe()

,D_47,B_5,R_24,D_65,B_19,D_119,S_5,B_14,S_26,D_91,B_9,R_16,D_81,D_133,P_4,B_37,S_12,D_89,D_61,D_122,S_25,R_8,R_20,D_92,B_20,D_144,D_70,B_27,D_48,target,B_40,B_22,S_13,B_25,D_114,B_33,B_4,S_15,D_74,S_23,S_11,B_7,D_143,S_6,D_93,D_131,D_121,R_25,B_3,D_120,D_39,R_14,D_113,D_115,D_140,R_15,D_55,D_118,D_69,B_12,P_2,D_44,S_24,R_28,S_27,B_24,D_80,D_68,B_6,B_11,B_30,R_1,S_8,R_22,D_124,S_20,B_26,D_129,D_125,D_127,D_86,B_8,R_13,D_128,R_4,D_62,D_59,S_16,D_103,D_83,D_60,D_71,S_18,B_16,B_32,D_54,B_10,R_6,D_84,B_31,B_21,R_5,R_2,R_12,R_23,R_19,B_23,R_10,R_17,R_21,D_102,B_13,B_28,B_2,D_45,D_130,D_139,R_27,D_141,B_18,D_117,S_22,D_107,S_3,D_51,D_52,B_36,D_41,B_41,D_145,D_116,S_19,D_72,R_3,D_79,D_109,D_78,D_112,R_7,D_96,D_123,D_75,B_38,D_126,D_46,S_7,P_3,R_11,D_58,D_94,S_17,D_104,B_1,B_15,R_18
count,49365.000000,49365.000000,4.936500e+04,4.936500e+04,4.936500e+04,49365.000000,49365.000000,49365.000000,4.936500e+04,4.936500e+04,4.936500e+04,4.936500e+04,4.936500e+04,4.936500e+04,4.936500e+04,49365.000000,49365.000000,4.936500e+04,49365.000000,4.936500e+04,49365.000000,4.936500e+04,4.936500e+04,4.936500e+04,4.936500e+04,4.936500e+04,4.936500e+04,4.936500e+04,49365.000000,49365.000000,4.936500e+04,4.936500e+04,4.936500e+04,49365.000000,49365.000000,4.936500e+04,4.936500e+04,49365.000000,49365.000000,49365.000000,49365.000000,49365.000000,4.936500e+04,4.936500e+04,4.936500e+04,4.936500e+04,49365.000000,4.936500e+04,4.936500e+04,49365.000000,4.936500e+04,4.936500e+04,4.936500e+04,49365.000000,4.936500e+04,4.936500e+04,49365.000000,4.936500e+04,4.936500e+04,4.936500e+04,49365.000000,4.936500e+04,49365.000000,4.936500e+04,4.936500e+04,4.936500e+04,4.936500e+04,49365.000000,49365.000000,49365.000000,49365.000000,4.936500e+04,49365.000000,4.936500e+04,49365.000000,4.936500e+04,4.936500e+04,4.936500e+04,4.936500e+04,4.936500e+04,4.936500e+04,4.936500e+04,4.936500e+04,49365.000000,4.936500e+04,49365.000000,49365.000000,4.936500e+04,49365.000000,4.936500e+04,4.936500e+04,49365.000000,4.936500e+04,4.936500e+04,4.936500e+04,49365.000000,49365.000000,4.936500e+04,4.936500e+04,49365.000000,4.936500e+04,4.936500e+04,4.936500e+04,49365.000000,4.936500e+04,4.936500e+04,49365.000000,4.936500e+04,4.936500e+04,4.936500e+04,4.936500e+04,49365.000000,49365.000000,49365.000000,49365.000000,4.936500e+04,4.936500e+04,49365.000000,4.936500e+04,49365.000000,49365.000000,49365.000000,4.936500e+04,49365.000000,4.936500e+04,49365.000000,4.936500e+04,4.936500e+04,4.936500e+04,4.936500e+04,49365.000000,4.936500e+04,4.936500e+04,4.936500e+04,4.936500e+04,4.936500e+04,4.936500e+04,4.936500e+04,4.936500e+04,4.936500e+04,4.936500e+04,4.936500e+04,49365.000000,49365.000000,49365.000000,49365.000000,49365.000000,4.936500e+04,4.936500e+04,4.936500e+04,4.936500e+04,4.936500e+04,49365.000000,49365.000000,4.936500e+04
mean,0.406627,0.082423,1.887674e-02,4.179396e-02,1.513353e-01,0.279385,0.085801,0.100383,6.838555e-02,5.978214e-02,1.911401e-01,5.893295e-02,5.560024e-02,4.481998e-02,1.446612e-01,0.123735,0.240575,6.493125e-03,0.426924,4.046082e-01,0.923865,3.976188e-02,3.842496e-02,9.036330e-02,2.378849e-01,5.368130e-02,1.082142e-01,5.004689e-03,0.370448,0.251372,1.827179e-01,1.070847e-01,2.543187e-01,0.103543,-30.963334,6.115911e-01,1.729004e-01,0.375682,0.153252,0.177616,0.375341,0.186608,1.790320e-01,2.438465e-01,1.474684e-02,1.005159e-01,0.533886,8.016705e-03,1.334118e-01,-31.446774,1.540732e-01,2.758980e-01,1.519643e-01,0.276974,2.617987e-02,1.997641e-02,0.295342,2.835521e-01,1.110583e-01,9.310653e-02,0.655293,1.135750e-01,0.736838,5.776375e-03,2.828892e-01,3.914361e-02,1.109415e-01,-33.693447,0.155604,0.111444,-0.269179,7.837190e-02,0.326606,8.573064e-03,0.303071,1.653307e-02,8.417950e-02,4.310990e-01,8.178455e-02,1.064360e-01,3.839755e-02,4.514750e-01,5.483498e-03,0.590570,3.164572e-02,0.175816,0.364209,4.591986e-02,0.460007,3.913885e-02,3.771932e-01,0.071569,3.187111e-02,3.504496e-01,2.662183e-02,0.983286,0.223226,5.824338e-02,4.670710e-02,0.996840,4.575187e-02,3.657684e-02,4.821153